# Using AdaptiveMasterMSM to run Gromacs
A **gro** file corresponding to alanine dipeptide is employed as example. Below the equilibration of the system is done automatically by the class Controller. Controller is also subsequently employed to run a typical MD simulation by using Gromacs.

In [1]:
from adaptivemastermsm import controller

We invoke **Controller** class for the **Equilibration** step, since later this will be used for production.

In [3]:
con = controller.Controller()
con.run_parameters('Equilibration')

In [6]:
# Note nvt.mdp and npt.mdp files must be in the working directory
# before this step
con.sys_equilibration('alaTB.gro', forcefield='amber03', water='tip3p')

gmx pdb2gmx -f alaTB.gro -o alaTB.gro -p topol.top -ff amber03 -water tip3p
gmx editconf -f alaTB.gro -o conf_edit.gro -c -d 1.000000 -bt cubic
gmx solvate -cp conf_edit.gro -cs spc216.gro -p topol.top -o conf_solv.gro
gmx grompp -f minimization.mdp -c conf_solv.gro -p topol.top -o minimization; gmx mdrun -v -s minimization -x minimization.xtc -deffnm minimization
gmx grompp -f nvt.mdp -c minimization.gro -r minimization.gro -p topol.top -o nvt.tpr; gmx mdrun -v -s nvt.tpr -deffnm nvt
gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr; gmx mdrun -v -s npt.tpr -deffnm npt


Once the simulation box is generated, the overall system is equilibrated, and a thermodynamically stable ensemble is obtained, we proceed to run the simulation.

In [4]:
con.run_parameters('Production')

In [5]:
con.sys_production('alaTB.gro', 'amber03', 'tip3p')

gmx grompp -c data/gro/processed/0.gro -p data/top/processed/0.top -f data/mdp/0.mdp -t npt.cpt -o data/tpr/0.tpr -maxwarn 1
gmx mdrun -v -s data/tpr/0.tpr -deffnm data/gro/processed/0.gro
out: b'' err: b"              :-) GROMACS - gmx mdrun, 2020.1-Ubuntu-2020.1-1 (-:\n\n                            GROMACS is written by:\n     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen\n    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    \n Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     \n  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  \n  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis\n    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    \n  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   \n    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   \n    Sander Pronk      Roland Schulz      Micha